# RTS dispatch Example

This notebook is intended to demonstrate some of the basic capability enabled by SIIP::Power. Specifically, this notebook gives some examples of how SIIP::Power can be used to conduct *Unit Commitment* on the RTS system



## Packages

 - *[PowerSystems.jl](https://github.com/NREL/PowerSystems.jl)* We take advantage of Julia's dynamic types and functional dispatch in our implementation of PowerSystems to define data schemas for Power Systems Analysis problems.

 - *[PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)* We leverage the schemas defined in PowerSystems to create functions for defining Power Systems Analysis Problems. 

## Package Setup

This notebook requires Julia 1.1 and uses the environment setup in the subfolder `env`. You can setup the environment with the next two cells.

The environment should look like:
```julia
      Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.5
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.18.0
  [e2685f51] ECOS v0.9.4
  [60bf3e95] GLPK v0.9.1
  [b6b21f68] Ipopt v0.5.4
  [4076af6c] JuMP v0.19.1
  [51fcb6bd] NamedColors v0.2.0
  [f0f68f2c] PlotlyJS v0.12.3+ #17b5821 (https://github.com/sglyon/PlotlyJS.jl.git)
  [e690365d] PowerSimulations v0.1.0 #master (https://github.com/nrel/PowerSimulations.jl)
  [bcd98974] PowerSystems v0.3.2
  [9e3dc215] TimeSeries v0.14.1
  [0f1e0344] WebIO v0.8.1+ #4f97d72 (https://github.com/JuliaGizmos/WebIO.jl.git)
```
Execute the next code block to activate the necessary environment.

In [1]:
] activate env; instantiate

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
] status

    Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.5
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.18.0
  [e2685f51] ECOS v0.9.4
  [60bf3e95] GLPK v0.9.1
  [b6b21f68] Ipopt v0.5.4
  [4076af6c] JuMP v0.19.1
  [51fcb6bd] NamedColors v0.2.0
  [f0f68f2c] PlotlyJS v0.12.3+ #17b5821 (https://github.com/sglyon/PlotlyJS.jl.git)
  [e690365d] PowerSimulations v0.1.0 #master (https://github.com/nrel/PowerSimulations.jl)
  [bcd98974] PowerSystems v0.3.2
  [9e3dc215] TimeSeries v0.14.1
  [0f1e0344] WebIO v0.8.1+ #4f97d72 (https://github.com/JuliaGizmos/WebIO.jl.git)


In [3]:
# Modeling Packages
using PowerSystems; # Power System Data Model
using PowerSimulations; # Power System Modeling
using JuMP;         # Optimization Modeling

### can use Xpress if you have a local license
#using Xpress;          # Optimization Solver
#optimizer = with_optimizer(Xpress.Optimizer, OUTPUTLOG=1,MIPTOL=0.00001)

### otherwise, use GLPK
using GLPK;
optimizer = with_optimizer(GLPK.Optimizer)


OptimizerFactory(GLPK.Optimizer, (), Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}())

In [4]:
const PSI = PowerSimulations;
const PSY = PowerSystems;

In [5]:
# Result Inspection Packages
using DataFrames;

# RTS Preloads

In [6]:

BASE_DIR = abspath(joinpath(dirname(Base.find_package("PowerSystems")), ".."))
DATA_DIR = joinpath(BASE_DIR, "data")
RTS_GMLC_DIR = joinpath(DATA_DIR, "RTS_GMLC")



"/Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC"

In [7]:

cdm_dict = PSY.csv2ps_dict(RTS_GMLC_DIR, 100.0)

# PowerSimulations.jl is under development, until it is finished, some system elements aren't supported
# for now, we'll remove them from the System.
delete!(cdm_dict,"dcline")
cdm_dict["gen"]["Renewable"]["PV"] = Dict{String,Any}()
cdm_dict["gen"]["Renewable"]["RTPV"] = Dict{String,Any}()
cdm_dict["gen"]["Renewable"]["WIND"] = Dict{String,Any}()
cdm_dict["gen"]["Hydro"] = Dict{String,Any}()
delete!(cdm_dict["forecast"]["DA"],"gen")
cdm_dict["forecast"]["DA"]["load"] = cdm_dict["forecast"]["DA"]["load"][1:2,:]
PSY.assign_ts_data(cdm_dict,cdm_dict["forecast"]["DA"])

sys_rts = PSY.System(cdm_dict)


┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:34
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 104_PV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 101_PV_2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 101_PV_3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 10

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 213_RTPV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/WIND/DAY_AHEAD_wind.csv for 309_WIND_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/WIND/DAY_AHEAD_wind.csv for 317_WIND_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/WIND/DAY

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 320_RTPV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 320_RTPV_2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 320_RTPV_3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/REAL_TIME_pv.csv for 102_PV_2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/REAL_TIME_pv.csv for 104_PV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/REAL_TIME_pv.csv for 101_PV_2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/REAL_TIME_pv.csv for 10

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 118_RTPV_10
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 213_RTPV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/WIND/REAL_TIME_wind.csv for 309_WIND_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/WIND/RE

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 313_RTPV_12
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 320_RTPV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 320_RTPV_2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/a865r/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/RE

┌ Warning: key 'dcline' not found in PowerSystems dictionary, this will result in an empty DCLines array
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/parsers/dict_to_struct.jl:52
┌ Error: Model doesn't contain a slack bus
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/utils/IO/system_checks.jl:51
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will trans

┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/a865r/src/u

System:
   buses: Bus[Bus(name="Abel"), Bus(name="Adams"), Bus(name="Adler"), Bus(name="Agricola"), Bus(name="Aiken"), Bus(name="Alber"), Bus(name="Alder"), Bus(name="Alger"), Bus(name="Ali"), Bus(name="Allen")  …  Bus(name="Chifa"), Bus(name="Chuhsi"), Bus(name="Clark"), Bus(name="Clay"), Bus(name="Clive"), Bus(name="Cobb"), Bus(name="Cole"), Bus(name="Comte"), Bus(name="Curie"), Bus(name="Curtiss")]
   generators: 
     GenClasses(T:73,R:0,H:0):
   thermal: ThermalDispatch[ThermalDispatch(name="322_CT_6"), ThermalDispatch(name="321_CC_1"), ThermalDispatch(name="202_STEAM_3"), ThermalDispatch(name="315_STEAM_1"), ThermalDispatch(name="223_CT_4"), ThermalDispatch(name="123_STEAM_2"), ThermalDispatch(name="213_CT_1"), ThermalDispatch(name="223_CT_6"), ThermalDispatch(name="313_CC_1"), ThermalDispatch(name="202_CT_2")  …  ThermalDispatch(name="216_STEAM_1"), ThermalDispatch(name="315_CT_6"), ThermalDispatch(name="201_CT_2"), ThermalDispatch(name="201_STEAM_3"), ThermalDispatch(name="201_

# RTS-GMLC

## Define the model

In [11]:
ED_copperplate = PSI.EconomicDispatch(sys_rts, PSI.CopperPlatePowerModel; optimizer = optimizer, parameters=false)

PowerSimulations.PowerOperationModel{EconomicDispatch,CopperPlatePowerModel}(EconomicDispatch, CopperPlatePowerModel, Dict{Symbol,PowerSimulations.DeviceModel}(:ThermalGenerators=>DeviceModel{ThermalGen,ThermalDispatch}(ThermalGen, ThermalDispatch),:RenewableGenerators=>DeviceModel{RenewableGen,RenewableFullDispatch}(RenewableGen, RenewableFullDispatch),:Loads=>DeviceModel{PowerLoad,StaticPowerLoad}(PowerLoad, StaticPowerLoad)), Dict{Symbol,PowerSimulations.DeviceModel}(:Lines=>DeviceModel{Branch,SeriesLine}(Branch, SeriesLine)), Dict{Symbol,PowerSimulations.ServiceModel}(:Reserves=>ServiceModel{Reserve,AbstractReservesForm}(Reserve, AbstractReservesForm)), System(buses:73,GenClasses(T:73,R:0,H:0),loads:73,branches:120,storage:1), PowerSimulations.CanonicalModel(A JuMP Model
Minimization problem with:
Variables: 584
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 438 constraints
`VariableRef`-in-`MathOptInterface.Le

## Run the Simulation

In [12]:
solve_op_model!(ED_copperplate)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/cbarrows/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


PowerSimulations.OpertationModelResults(Dict(:Pth=>2×73 DataFrame. Omitted printing of 68 columns
│ Row │ 322_CT_6 │ 321_CC_1 │ 202_STEAM_3 │ 315_STEAM_1 │ 223_CT_4 │
│     │ Float64  │ Float64  │ Float64     │ Float64     │ Float64  │
├─────┼──────────┼──────────┼─────────────┼─────────────┼──────────┤
│ 1   │ NaN      │ NaN      │ NaN         │ NaN         │ NaN      │
│ 2   │ NaN      │ NaN      │ NaN         │ NaN         │ NaN      │), Dict(:ED=>4.5139e5), Dict{Symbol,Any}(:dual_status=>NO_SOLUTION,:primal_status=>NO_SOLUTION,:termination_status=>INFEASIBLE_OR_UNBOUNDED))

The model is `INFEASIBLE` because we are attempting to run an `EocnomicDispatch` with all generators in the RTS committed and the system is constrained by minimum generation levels that are too high to meet low-load conditions. We can resolve this by relaxing thermal minimum generation levels.

In [13]:
ED_copperplate.devices[:ThermalGenerators] = PowerSimulations.DeviceModel(ThermalGen, PSI.ThermalDispatchNoMin)
PSI.build_op_model!(ED_copperplate; optimizer = optimizer, parameters=false)

In [14]:
solve_op_model!(ED_copperplate)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/cbarrows/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


PowerSimulations.OpertationModelResults(Dict(:Pth=>2×73 DataFrame. Omitted printing of 68 columns
│ Row │ 322_CT_6 │ 321_CC_1 │ 202_STEAM_3 │ 315_STEAM_1 │ 223_CT_4 │
│     │ Float64  │ Float64  │ Float64     │ Float64     │ Float64  │
├─────┼──────────┼──────────┼─────────────┼─────────────┼──────────┤
│ 1   │ 0.33     │ 1.23     │ 0.46        │ 0.0         │ 0.11     │
│ 2   │ 0.33     │ 1.23     │ 0.46        │ 0.0         │ 0.11     │), Dict(:ED=>3.68007e5), Dict{Symbol,Any}(:dual_status=>FEASIBLE_POINT,:primal_status=>FEASIBLE_POINT,:termination_status=>OPTIMAL))